In [1]:
#导入库
import pandas as pd  #导入pandas库
import jieba    #导入结巴分词库
import os    #导入os库
import re
import markdown    #导入markdown库
from gensim.models.doc2vec import Doc2Vec, TaggedDocument   #导入Doc2Vec模型和TaggedDocument类

In [2]:
# 新词文件的路径
newwords_list = 'newwords.txt'  # 替换为你的文件路径

# 从文件中读取新词并添加到jieba字典中
with open(newwords_list, 'r', encoding='utf-8') as file:
    for word in file:
        jieba.add_word(word.strip())

Building prefix dict from the default dictionary ...


Dumping model to file cache C:\Users\Parejo\AppData\Local\Temp\jieba.cache
Loading model cost 0.760 seconds.
Prefix dict has been built successfully.


In [4]:
#按行读取文件，返回文件的行字符串列表
def read_file(file_name):                                                                   #file_name为文件路径
    fp = open(file_name, "r", encoding="utf-8")                                              #设置文件对象
    content_lines = fp.readlines()                                                            #直接将文件中按行读到list里，效果与方法2一样
    fp.close()                                                                                 #关闭文件

                                                         #去除行末的换行符，否则会在停用词匹配的过程中产生干扰
    for i in range(len(content_lines)):                                                        #遍历每一行
        content_lines[i] = content_lines[i].rstrip("\n")                                       #去除换行符
    return content_lines                                                                       #返回文件的行字符串列表

def clean_stopword(str, stopword_list):                                                         # str为待分词的字符串，stopword_list为停用词列表
    result = []                                                                                 #用于存放分词结果
    word_list = jieba.lcut(str)                                                                 # 分词后返回一个列表  jieba.cut(）   返回的是一个迭代器
    for w in word_list:                                                                         #遍历每个词
        if w not in stopword_list and w != '\n' and w != '\u3000' and  w != '\u2002':                              #去除停用词和换行符
            result.append(w)                                                                    #将不在停用词表中的词添加到列表中
    return result                  

In [6]:
stopword_list = read_file('stopwords.txt')                                                   #读取停用词表
data = []                                                                                    #用于存放所有文档的分词结果
file_list = []
file_walk = os.walk('THUCNews')
cnt = 0
for i in range(3):
    print(i)
    if i == 0:
        next(file_walk)
    else:
        walk_paths = next(file_walk)
        file_paths = walk_paths[2]
        file_paths.sort()
        for file in file_paths:
            file_list.append(walk_paths[0]+'/'+file)
            with open(file_list[-1], encoding='utf-8') as f:
                text = f.read()                                             #读取文件内容
                doc_list = clean_stopword(text, stopword_list)              #清洗停用词
                doc = TaggedDocument(doc_list, tags=[cnt])                  #将文档转换为TaggedDocument类型
                cnt += 1
                data.append(doc)                                            #将文档添加到data列表中



0
1


KeyboardInterrupt: 

In [7]:
# 训练模型
model = Doc2Vec(dm =1, vector_size=300, window=8, seed=1, workers=1)
model.build_vocab(data)                                                                          #建立词汇表
model.train(data, total_examples=model.corpus_count, epochs=model.epochs) 

In [8]:
# assessing the model
ranks = []
second_ranks = []
for doc_id in range(6000):
    inferred_vector = model.infer_vector(data[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [9]:
import collections

counter = collections.Counter(ranks)
print(counter)

Counter({0: 5599, 1: 84, 2: 23, 3: 21, 4: 16, 5: 11, 6: 10, 9: 8, 7: 7, 8: 6, 13: 6, 15: 5, 14: 5, 12: 4, 25: 4, 11: 4, 24: 3, 109: 3, 18: 3, 20: 3, 23: 3, 33: 3, 28: 3, 38: 3, 41: 2, 16: 2, 529: 2, 22: 2, 276: 2, 102: 2, 65: 2, 318: 2, 56: 2, 252: 2, 17: 2, 69: 2, 919: 2, 46: 2, 1263: 1, 986: 1, 9192: 1, 39947: 1, 47: 1, 261: 1, 870: 1, 138: 1, 2139: 1, 3933: 1, 171: 1, 1158: 1, 1199: 1, 74: 1, 782: 1, 284: 1, 172: 1, 26: 1, 885: 1, 664: 1, 306: 1, 304: 1, 75: 1, 1879: 1, 6444: 1, 96: 1, 1088: 1, 901: 1, 1045: 1, 613: 1, 1582: 1, 1001: 1, 1411: 1, 430: 1, 187: 1, 1755: 1, 342: 1, 84: 1, 1841: 1, 606: 1, 843: 1, 420: 1, 63: 1, 2194: 1, 44: 1, 2547: 1, 58: 1, 36819: 1, 406: 1, 583: 1, 21: 1, 321: 1, 5651: 1, 213: 1, 39404: 1, 132: 1, 5275: 1, 85: 1, 253: 1, 45: 1, 383: 1, 6876: 1, 1171: 1, 2167: 1, 32: 1, 447: 1, 249: 1, 449: 1, 326: 1, 40: 1, 29: 1, 5079: 1, 12473: 1, 5396: 1, 42029: 1, 1279: 1, 239: 1, 330: 1, 24999: 1, 290: 1, 43: 1, 661: 1, 1163: 1, 165: 1, 127: 1, 143: 1, 30: 1, 14

In [10]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(data[doc_id].words)))   
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(data[sims[index][0]].words)))

Document (5999): «中宣部 部长 刘云山 内蒙 满洲里市 调研 中广网 呼和浩特 月 日 消息 记者 陈青平   满洲里 电台 崔艳凤   通讯员 张鹏   月 31 日 中共中央政治局 委员 中央书记处 书记 中宣部 部长 刘云山 来到 内蒙古 满洲里市 视察 调研 满洲里市 期间 刘云山 一行 视察 市 国门 景区 红色 展厅 新 公路 口岸 新闻 演播 大厅 俄罗斯 艺术 博物馆 满洲里 学院 木材 交易 展厅 视察 刘云山 充分肯定 我市 经济社会 发展 特别 宣传 思想 文化 工作 成绩 说 满洲里市 经济繁荣 社会 政治 稳定 民族团结 和睦 人民 安居乐业 充满生机 活力 呈现出 科学 发展 和谐 发展 文明 发展 态势 自治区 党委 政府 正确 领导 满洲里市 抓 经济社会 发展 文化 建设 高度重视 高度 自觉 宣传 思想 文化战线 在实践中 创造 经验 特色 成效 亮点 独特 文化 资源 依托 俄罗斯 蒙古国 广泛开展 文化 交流活动 全方位 对外 展示 生态 满洲里 开放 满洲里 文化 满洲里 发展 满洲里 良好形象 城市 建设 满洲里 紧紧抓住 地缘文化 优势 俄蒙 多种 建筑 元素 巧妙 城市 建设 富有 文化 内涵 刘云山 指出 满洲里 中国共产党 早期 一座 红色 国际 桥梁 我党 历史 作用 满洲里市 发展 红色旅游 定位 准确 红色 内容 旅游 载体 宣传 革命 传统 革命 历史 新 时期 思想 政治 教育 工作 一大 创新 旅游业 拓展 发展 红色旅游 思想 教育 工作 兼顾 旅游业 发展 规律 不断丰富 挖掘 红色 内容 拓展 创新 旅游 这一 载体 紧密结合 吸引 更 游客 关注 参与 红色旅游 发展 红色旅游 意义 落到实处 满洲里市 红色旅游 资源 优势 善于 挖掘 红色 资源 深刻 内涵 拓展 创新 发展 国内 红色旅游 还应 发展 俄罗斯 红色旅游 多方 收集 历史 资料 实物 精心策划 布展 方案 红色旅游 项目 相比 主题 新颖 利益 独特 视角 广阔 意义 深远 力争 全国 红色旅游 发展 起到 示范作用 红色旅游 全国 更 快 更好 发展 刘云山 指出 满洲里 地处 祖国 边疆地区 俄罗斯 蒙古 各项 交往 紧密 做好 精神文明 建设 工作 满洲里市 建设 社会主义 核心 价值体系 广泛开展 道

In [11]:
# Pick a random document from the corpus and infer a vector from the model
import random
doc_id = random.randint(0, 6000 - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(data[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(data[sim_id[0]].words)))

Train Document (2831): «胡锦涛 会见 武警部队 人员 服务大局 新华网 北京 月 日电   记者 曹智 张东波 中共中央 总书记 国家 主席 中央军委 主席 胡锦涛 日 会见 武警部队 党委 一届 六次 全体会议 人员 时 高举 中国 特色 社会主义 伟大旗帜 邓小平理论 三个代表 思想 指导 贯彻落实 科学 发展观 自觉 服从 服务 党和国家 工作 大局 忠实 履行 武警部队 职责 使命 优异成绩 迎接 新 中国 成立 60 周年   中共 中央政治局常委 中央 政法委 书记 周永康 参加 会见 胡锦涛 代表 党中央 国务院 中央军委 出席会议 同志 武警部队 官兵 致以 诚挚 问候 说 一年 武警部队 党委 坚决贯彻 党中央 国务院 中央军委 决策 指示 带领 官兵 顽强拼搏 开拓进取 部队 建设 新 圆满完成 党和人民 赋予 各项任务 特别 急难险重 发挥 作用 作出 突出贡献 党 政府 人民 感谢 胡锦涛 指出 2009 年 党和国家 事业 发展 进程 关键 一年 面临 发展 机遇 面临 严峻 挑战 繁重 艰巨 武警部队 思想 政治 建设 中国 特色 社会主义 理论体系 武装 官兵 培育 当代 革命军人 核心 价值观 始终保持 部队 建设 正确 政治 方向 抓 军事训练 增强 遂行 执勤 处突 反恐 多样化 能力 实际行动 捍卫 国家 统一 维护 社会 和谐 稳定 保障 人民 安居乐业 依法 治警 治警 扎实 做好 抓 基层 打基础 工作 提高 部队 管理 教育 水平 党组织 能力 建设 先进性 建设 充分发挥 党 政治 优势 组织 优势 带领 官兵 夺取 建设 小康社会 新 胜利 作出 更大 贡献 中共中央政治局 委员 中央军委 副 主席 郭伯雄 中共中央政治局 委员 中央军委 副 主席 徐才厚 中央军委 委员 国务委员 兼 国防部长 梁光烈 国务委员 公安部 部长 兼 武警部队 第一 政委 孟建柱 参加 会见 中央军委 委员 陈炳德 李继耐 廖锡龙 常万全 靖志远 吴 胜利 许其亮 参加 会见»

Similar Document (60240, 0.7989599704742432): «组图 奥巴马 参观 二战 集中营 纪念碑 月 日 美国 总统 奥巴马 当日 来到 位于 魏玛 布痕 瓦尔德 集中营 纪念碑 参观 纪念碑 献花

## 比较多个文档的相似性

In [1]:
import os
test_data = []
test_file_paths = os.listdir('【文本】数据治理机构')
for i, file in enumerate(test_file_paths):
    with open('【文本】数据治理机构/'+file) as f:
        text = f.read()
        html = markdown.markdown(text)
        doc = clean_stopword(html, stopword_list)
        test_data.append(doc)

NameError: name 'os' is not defined

In [ ]:
## 模型测试


In [13]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_data) - 1)
inferred_vector = model.infer_vector(test_data[doc_id])   # 通过训练好的模型，将测试集中的文档转换为向量
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))   # 与训练集中的所有文档进行比较，返回相似度最高的前n个文档


# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_data[doc_id])))   # 打印测试集中的文档
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)      # 打印模型信息
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:   
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(data[sims[index][0]].words)))       

Test Document (48): «组织 起草 全市 政务服务 政务信息化 相关 政策 标准规范 组织 实施 统筹 推进 全市 智慧城市 建设 拟订 建设 规划 年度 建设 计划 组织 实施 统筹 推进 全市 一网通办 前提 最多跑一次 改革 工作 统筹 市级 政务信息系统 建设 规划 提出 项目 建设 具体意见 统筹 管理 政务云平台 政务 服务平台 银企 信息 共享 融资 服务平台 电子政务 网络 统筹 推进 全市 政务服务 数据管理 体系 建设 指导 全市 政务服务 数据管理 工作 组织协调 全市 政务服务 环境 优化 评价 工作 负责 市级 政务 服务质量 监督管理 负责 全市 行政审批 制度 改革 审批服务 便民化 相关 工作 负责 全市 政务服务 事项 目录 管理 标准化 建设 统筹 全市 数据资源管理 建设 工作 负责 政务数据资源 采集 分类 管理 分析 工作 承担 大数据 人才队伍 建设 工作 协调 服务 数据产业 发展 统筹 全市 电子政务 基础设施 信息系统 数据资源 保障 工作 负责 智慧城市 平台 技术 运营 体系 建设 监督管理 市级 信息系统 数据库安全 负责 指导 规范 监督 全市 政务服务 实体 大厅 工作 负责 市 行政 服务中心 各县 区 行政 审批服务 效能 日常 监管 工作 做好 审批服务 事项 进驻 行政 服务中心 大厅 相关 工作 负责 市政府 提出 设立 调整 市政府 部门 办事大厅 窗口 意见 建议 负责 全市 12345 政务服务 热线 平台 建设 管理工作 承担 商丘市 人民政府 推进 政府职能 协调 小组 商丘市 深化 放管服 改革 推进 审批服务 便民化 领导小组 日常 工作 市委 市政府 交办»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec<dm/m,d300,n5,w8,mc5,s0.001>:

MOST (22611, 0.7025723457336426): «巴基斯坦 拉瓦尔品第 市 发生爆炸 死亡 新华网 快讯 巴基斯坦 首都 伊斯兰堡 拉瓦尔品第 市 日晚 发生爆炸 至少 死亡 受伤»

MEDIAN (25068, 0.390019029378891): «韩国 称 接受 韩 开城 协议 无效 环球时报 特约记者 王斯 报道   韩国政府 日 朝

In [14]:
n = len(test_file_paths)
with open('similarity_on_test_data.txt', 'w') as f:
    for i in range(n):
        for j in range(i+1, n):
            f.write(f"{test_file_paths[i]}, {test_file_paths[j]}, {model.similarity_unseen_docs(test_data[i], test_data[j]):.4f}")   # 计算测试集中两两文档的相似度
            f.write('\n')


In [15]:
n = len(test_file_paths)
repeat_times = 100  # 相似度计算的重复次数

with open('similarity_on_test_data_avg.txt', 'w') as f:
    for i in range(n):
        for j in range(i+1, n):
            # 初始化相似度总和变量
            similarity_sum = 0
            
            # 重复计算相似度100次
            for _ in range(repeat_times):
                similarity = model.similarity_unseen_docs(test_data[i], test_data[j])  # 计算相似度
                similarity_sum += similarity

            # 计算相似度的平均值
            similarity_avg = similarity_sum / repeat_times

            # 将平均相似度写入文件
            f.write(f"{test_file_paths[i]}, {test_file_paths[j]}, {similarity_avg:.4f}\n")

In [24]:
# 假设 test_data 已经包含了一系列文档，test_file_paths 包含了文件名
test_file_paths = os.listdir('【文本】数据治理机构')

# 提取每个文件名前的数字，并转换为整数
numbers = [int(re.match(r'(\d+)', file_name).group()) for file_name in test_file_paths]

# 根据提取的数字对 test_file_paths 和 test_data 同时排序
sorted_files_and_data = sorted(zip(numbers, test_file_paths, test_data))

# 用于存储最终输出的字符串
output = ""

# 遍历排序后的文件和数据
for num, file_name, doc in sorted_files_and_data:
    # 使用文件名作为序号
    document_with_number = f"序号 {file_name}:\n{doc}\n\n"
    
    # 将每个处理后的文本添加到输出字符串
    output += document_with_number

print(output)

序号 1北京市2018.md:
负责 研究 提出 本市 数据管理 规范 技术标准 建议 负责 本市 政务数据 相关 社会 数据 汇聚 管理 共享 开放 评估 负责 市级 政务云 数据管理 平台 数据 基础设施 建设 运 维和 支撑 负责 社会 信用 数据 应用服务 承担 社会 信用 体系 建设 辅助性 事务性 工作 负责 互联网+政务服务 信息化 基础 支撑 平台 建设 运 维和 保障 承担 政府 投资 信息化 项目 技术性 审核 支撑 服务 工作 承担 组织 信息化 项目 技术论证 评估 验收 支撑 服务 工作

序号 2天津市2018.md:
参与 拟定 大数据 发展 规划 信息化 建设 标准 信息 数据 标准规范 承担 信息化 项目 建设 组织 实施 运行 保障 工作 承担 市 政务云 平台 全市 信息资源 统一 共享 交换 平台 平台 建设 运行 管理 日常 维护 工作 承担 统一 信息 资源库 建设 承担 大数据 全 业态 发展 工作 宏观经济 监测 预警 提供 决策 支持 信息 咨询服务 承担 市级 电子政务 网络 运维 管理工作 承担 信息 数据安全 保障 工作

序号 3河北省石家庄市2019.md:
贯彻执行 国家 省 电子政务 智慧城市 政务服务 数据资源管理 方针政策 法律法规 起草 相关 地方性 法规 政府 规章 草案 拟订 全市 数据资源管理 发展 规划 年度计划 政策措施 评价 体系 组织 实施 组织 实施 国家 地方 数据 技术标准 研究 制定 我市 数据资源 采集 存储 开放 共享 标准规范 监督 实施 会同 政务服务 管理 部门 做好 互联网+政务服务 工作 规划 项目 建设 工作 负责 在线 政务 服务平台 建设 推进 在线 政务 服务平台 规范化 标准化 集约化 建设 拟订 全市 政务数据 公共数据资源 目录 制定 归集 管理 整理 利用 共享开放 管理 办法 组织 实施 推动 数据资源 政府 管理 社会 治理 领域 负责 市 电子政务 组织 推动 工作 负责 市 政务 网络 建设 管理 党政机关 信息化 建设 指导 负责 市级 智慧城市 项目 建设 数据资源 基础设施 建设 管理 提出 部门 统筹安排 市级 信息化 项目 资金 建议 负责 全市 数据资源 政务信息化 领域 对外 交流 合作 会同 部门 做好 全市 数据资源 政

In [28]:
# 将输出写入到一个文件
with open('数据治理机构文本预处理.txt', 'w', encoding='utf-8') as f:
    f.write(output)